<a href="https://colab.research.google.com/github/Madhugandha11/SpeechRecognition/blob/main/SpeechRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Using AssemblyAI to transcribe audio
API_KEY_ASSEMBLYAI = "f98ac765b67d48e6b2eedbfda7afc29e"

In [ ]:
import requests
filename = "/content/the_dog_story.m4a" #Audio to be transcribed

upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"

headers = {'authorization':API_KEY_ASSEMBLYAI }
#hearders here are used for authentication

#Uploading Local File for Trancription
def upload(filename):
  #Function to read the audiofile. Assembly AI reads files in chunks. Here, a chunk is pf 5242880 bytes.
  def read_file(filename, chunk_size=5242880): 
            with open(filename, 'rb') as _file:
                while True:
                    data = _file.read(chunk_size)
                    if not data:
                        break
                    yield data

        
  upload_response = requests.post(upload_endpoint,
                                headers=headers,
                                data=read_file(filename))

     

  audio_url = upload_response.json()['upload_url'] #Uploaded the file on Assembly AI.
  return(audio_url)

        

In [ ]:
#Trnacripting the audio file

def transcribe(audio_url ):
  transcript_request = { "audio_url": audio_url , "disfluencies": True}
  #Disfluencies here is used to include Filler words in our text. 

  transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
  
  job_id = transcript_response.json()['id']
  return job_id


In [ ]:
#Poll the Assembly AI to know if the trannscription is completed or in progress.

def poll(transcript_id):
    polling_endpoint = transcript_endpoint + '/' + transcript_id #url through which we will ask Assembly Ai the status
    polling_response = requests.get(polling_endpoint, headers=headers)
    return polling_response.json()


In [ ]:
def get_transcription_result_url(url):
    transcribe_id = transcribe(url)
    while True:
        data = poll(transcribe_id)
        if data['status'] == 'completed': #Checking the status of Transcription
            return data, None
        elif data['status'] == 'error': #Checking if there is an error or not
            return data, data['error']
            
        

In [ ]:
def save_transcript(url,filename):
    data, error = get_transcription_result_url(url)
    
    if data:
        filename = filename + '.txt'
        with open(filename, 'w') as f:
            f.write(data['text'])
        
        print('Transcript saved')
    elif error:
        print("Error!!!", error)

In [ ]:
audio_url = upload(filename)
save_transcript(audio_url, filename)

In [ ]:
file = "/content/the_dog_story.m4a"
content = open(file, "r")


FileContent = content.read()
print(FileContent)

In [ ]:
filler_word = ['uh', 'um', 'uhuh', 'hmm' ,'mhm']
list_of_filler_words_used=[]


with open("internship.m4a.txt", "r") as file:
    for line_number, line in enumerate(file, start=1):  
      for i in filler_word:
        if i in line:
          list_of_filler_words_used.append(i)
          
          
print(list_of_filler_words_used)
        

['uh', 'um']
